In [1]:
import os
import re
import requests
from gtts import gTTS
from urllib.parse import quote
from PIL import Image, ImageFilter
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips, VideoFileClip
import gradio as gr

In [2]:
# Dossiers
os.makedirs("images", exist_ok=True)
os.makedirs("voix", exist_ok=True)

In [3]:
# Fonctions
def flouter_bas_image(image_path, ratio=0.12):
    try:
        img = Image.open(image_path)
        largeur, hauteur = img.size
        hauteur_flou = int(hauteur * ratio)
        zone_flou = (0, hauteur - hauteur_flou, largeur, hauteur)
        bande = img.crop(zone_flou).filter(ImageFilter.GaussianBlur(radius=12))
        img.paste(bande, zone_flou)
        img.save(image_path)
    except Exception as e:
        print(f"Erreur floutage : {e}")

def extraire_mots_cles(texte, n=10):
    mots = re.findall(r'\b\w+\b', texte.lower())
    frequences = {}
    for mot in mots:
        if mot not in {"le", "la", "les", "un", "une", "de", "des", "et", "en", "je", "tu", "il", "elle"}:
            frequences[mot] = frequences.get(mot, 0) + 1
    mots_cles = sorted(frequences, key=frequences.get, reverse=True)[:n]
    return mots_cles

def generate_images_from_text(texte, nb_images=10):
    mots_cles = extraire_mots_cles(texte, n=nb_images)
    prompts = [texte + " " + mot for mot in mots_cles]
    image_paths = []
    for i, prompt in enumerate(prompts):
        try:
            encoded_prompt = quote(prompt)
            img_url = f"https://image.pollinations.ai/prompt/{encoded_prompt}"
            response = requests.get(img_url)
            response.raise_for_status()
            image_path = f"images/image{i+1}.png"
            with open(image_path, "wb") as f:
                f.write(response.content)
            flouter_bas_image(image_path)
            image_paths.append(image_path)
        except Exception as e:
            print(f"Erreur image {i+1} : {e}")
    return image_paths

def generate_audio(texte):
    path = "voix/narration.mp3"
    tts = gTTS(texte, lang="fr")
    tts.save(path)
    return path

def create_video(images, audio_path=None, duration_total=30):
    clips = []
    duration_par_image = duration_total / len(images)
    for img in images:
        clip = ImageClip(img).set_duration(duration_par_image)
        clips.append(clip)
    video = concatenate_videoclips(clips)
    if audio_path:
        audio = AudioFileClip(audio_path).subclip(0, video.duration)
        video = video.set_audio(audio)
    output_path = "video_finale.mp4"
    video.write_videofile(output_path, fps=24, verbose=False, logger=None)
    return output_path

def traitement_video(texte, ajouter_narration):
    try:
        images = generate_images_from_text(texte, nb_images=10)
        audio_path = generate_audio(texte)
        video_path = create_video(images, audio_path if ajouter_narration else None)
        return video_path, audio_path, audio_path
    except Exception as e:
        return f"Erreur : {e}", None, None

def generer_texte_depuis_prompt(prompt):
    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={'model': 'mistral', 'prompt': prompt, 'stream': False}
        )
        return response.json()['response']
    except Exception as e:
        return f"Erreur génération texte : {e}"

In [4]:
# Interface Gradio
with gr.Blocks(css="footer {display: none !important}") as interface:
    gr.Markdown("""
    <h1 style="text-align: center; color: #2D3748;"> Générateur Vidéo AI</h1>
    <p style="text-align: center; color: #4A5568;">
        Transforme tes pensées en une vidéo de 30 secondes avec des images artistiques, une narration douce et une interface fluide.
    </p>
    """)

    with gr.Tab("Générer un texte avec un prompt"):
        prompt_input = gr.Textbox(label="Prompt de texte", lines=4, placeholder="Décris ton idée ici…")
        bouton_generer_texte = gr.Button("Générer le texte")
        texte_genere = gr.Textbox(label="Texte généré", lines=10)

    with gr.Tab("Créer la vidéo"):
        texte_input = gr.Textbox(label="Ton texte", lines=10, placeholder="Colle ici ton texte ou utilise celui généré ci-dessus")
        narration_checkbox = gr.Checkbox(label="Ajouter la narration à la vidéo ?", value=True)
        bouton_generer_video = gr.Button("Générer la vidéo")

        with gr.Row():
            video_output = gr.Video(label="Vidéo finale")
            audio_output = gr.Audio(label="Écouter la narration")

        telecharger_audio = gr.File(label="Télécharger narration")
        telecharger_video = gr.File(label="Télécharger vidéo")

    # Lien entre actions
    bouton_generer_texte.click(fn=generer_texte_depuis_prompt, inputs=prompt_input, outputs=texte_genere)
    bouton_generer_video.click(
        fn=traitement_video,
        inputs=[texte_input, narration_checkbox],
        outputs=[video_output, audio_output, telecharger_audio]
    )
    bouton_generer_video.click(lambda: "video_finale.mp4", None, telecharger_video)

interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
